# 第２章 物体検出（SSD）　

## 2-1 物体検出とは

### 物体検出の概要

- 物体検出とは  
    １枚の画像に含まれている複数の物体に対して、物体の領域と物体名を特定するタスク。

- バウンディングボックス（Bounding box）  
    物体の位置を示す枠

- confidence  
    検出の確信度（１が一番大きい）

### 物体検出タスクのインプットとアウトプット

- インプット：画像  
    
    
- アウトプット  
    - バウンディングボックスの位置と大きさ情報
    - 物体のラベル情報
    - 検出の信頼度＝confidence


- ssdアルゴリズム   
    - バウンディングボックスの形  
        中心のx座標をcx、中心のy座標をcy、幅w、高さh
        
    - ラベル情報
        クラスの数＋背景１

### VOCデータセットとは

- 使用するデータセット
    - VOC２０１２データセット。
    - クラス２０種類（＋背景１）、訓練データ5717枚、検証データ5823枚


- アノテーションデータ
    - 画像ごとにxml形式のファイルで提供されている。
    - 長方形の左端のxmin座標、上端のymin座標、右端のxmax座標、下端のymax座標。

### SSDによる物体検出の流れ

- 流れ  
    1. 画像をリサイズ
    2. デフォルトボックス8732個を容易
    3. 画像をSSDのネットワークに入力
    4. 信頼度上位のデフォルトボックスを抽出
    5. オフセット情報による修正と被りの除去
    6. 一定の信頼度以上のものを最終出力に


- SSD300／SSD512  
    入力画像を300ピクセル／512ピクセルにリサイズする。

- デフォルトボックス  
    あらかじめ容易してある定型的な長方形
    
- オフセット情報  
    この長方形がどう変化すればバウンディングボックスになるのかという出力情報。  
    $(\Delta cx, \Delta cy, \Delta w, \Delta h )$
    
- バウンディングボックスの情報  
    $cx = cx\_d(1+0.1\Delta cx)$    
    $cy = cy\_d(1+0.1\Delta cy)$  
    $w = w\_d\times \exp(0.2\Delta w)$  
    $h = h\_d\times \exp(0.2\Delta h)$  

## 2-2 Datasetの実装

In [1]:
import os.path as osp
import random
import xml.etree.ElementTree as ET

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.utils.data as data

%matplotlib inline

In [2]:
# シードの固定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

### 画像データ、アノテーションデータへのファイルパスのリストを作成

In [3]:
# 学習、検証の画像データとアノテーションデータへのファイルパスリストを作成する。

def make_datapath_list(rootpath):
    '''
    データへのパスを格納したリストを作成する。
    
    Parameters
    ----------
    rootpath : str
        データフォルダへのパス
    
    Returns
    -------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        データへのパスを格納したリスト
    '''
    
    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    imgpath_template = osp.join(rootpath, 'JPEGImages', '%s.jpg')
    annopath_template = osp.join(rootpath, 'Anotations', '%s.xml')
    
    # 訓練と検証、それぞれのファイルのID（ファイル名）を取得する
    train_id_names = osp.join(rootpath + 'ImageSets/Main/train.txt')
    val_id_names = osp.join(rootpath + 'ImageSets/Main/val.txt')
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスリストを作成
    train_img_list = list()
    train_anno_list = list()
    
    for line in open(train_id_names):
        file_id = line.strip() # 空白スペースと改行を削除
        img_path = (imgpath_template % file_id) # 画像のパス
        anno_path  = (annopath_template % file_id) # アノテーションのパス
        train_img_list.append(img_path) # リストに追加
        train_anno_list.append(anno_path) # リストに追加
        
    # 検証データの画像ファイルとアノテーションファイルへのパスリストを作成
    val_img_list = list()
    val_anno_list = list()
    
    for line in open(val_id_names):
        file_id = line.strip() # 空白スペースと改行を削除
        img_path = (imgpath_template % file_id) # 画像のパス
        anno_path = (annopath_template % file_id) # アノテーションのパス
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list

In [4]:
# ファイルパスへのリストを作成
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

In [5]:
# 動作確認
print(train_img_list[0])

./data/VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg


### xml形式のアノテーションデータをリストに変換

-　xmlデータ形式のアノテーションデータをPythonのリスト型に変換する

- [xml.etree.ElementTree](https://docs.python.org/ja/3/library/xml.etree.elementtree.html)  
    ファイルを読み込むことでxmlデータをインポートすることができる。
- [.xml【拡張子】とは](https://wa3.i-3-i.info/word11856.html)
- [ルート要素](https://www.atmarkit.co.jp/aig/01xml/rootelement.html)
- []()

In [ ]:
# xml形式のアノテーションをリスト形式に変換するクラス

class Anno_xml2list(object):
    '''
    １枚の画像に対する「xml形式のアノテーションデータ」を、画像サイズで規格化してからリスト形式に変換する。
    
    Attributes
    ----------
    classes : リスト
        VOCのクラス名を格納したリスト
    '''
    
    def __init__(self, classes):
        
        self.classes = classes
    
    def __call__(self, xml_path, width, height):
        '''
        1枚の画像に対する「xml形式のアノテーションデータ」を、
        画像サイズで規格化してからリスト形式に変換する。
        
        Parameters
        ----------
        xml_path : str
            xmlファイルへのパス
        width : int
            対象画像の幅
        height :　int
            対象画像の高さ
        
        Returns
        -------
        ret : [[xmin, ymin, xmax, ymax, label_ind], ...]
            物体のアノテーションデータを格納したリスト。
            画像内に存在する物体数文のだけ要素を持つ。
        '''
        
        # 画像内のすべての物体のアノテーションリストをこのリストに格納する。
        ret = []
        
        # xmlファイルを読み込む
        xml = ET.parse(xml_path).getroot()
        
        # 画像内にある物体（object）の数だけループする
        for obj in xml.iter('object'):
            
            # アノテーションで検知がdifficultに設定されているものは除外
            difficult = int(obj.find('difficult').text())
            
            # 一つの物体に対するアノテーションを格納するリスト
            bindbox = []
            
            name = obj.find('name').text.lower().strip() # 物体名
            bbox = obj.find('bndbox') # バウンディングボックスの情報
            
            # アノテーションの xmin, ymim, xmax, ymax を取得し、0~1に規格化
            pts = ['xmin', 'ymin', 'xmax', 'ymax']
            
            for pt in (pts):
                # VOCは原点が(1, 1)なので1を引き算して(0, 0)に
                cur_pixel = int(bbox.find(pt).text) - 1
                
                # 幅、高さで規格化
                if pt == 'xmin' or pt == 'xmax': # x方向のときは幅で割り算
                    cur_pixel /= width
                else: # y方向のときは高さで割り算